# カスタムコンテナハンズオン　part2


# Part 2: カスタムコンテナを使って Amazon SageMakerで学習と推論をする


## 環境セットアップ

S3バケットとロールを準備します。

In [5]:
# S3 prefix
prefix = 'LAB-scikit-iris'

# Define IAM role
import boto3
import re

import os
import numpy as np
import pandas as pd
from sagemaker import get_execution_role

role = get_execution_role()

## SageMakerのセッションを取得

SageMakerを操作するためのセッションを取得します。

In [6]:
import sagemaker as sage
from time import gmtime, strftime

sess = sage.Session()

## 学習データをS3にアップロード

SageMaker の学習時に利用するデータは、S3 に置く必要があります。ローカルの data ディレクトリにあるデータをS3にアップロードします。
デフォルトでは SageMaker は sagemaker-{region}-{your aws account number} というバケットを使用します。当該バケットがない場合には、自動で新しく作成します。upload_data() メソッドの引数に bucket=XXXX という形でデータを配置するバケットを指定することも可能です。

In [9]:
WORK_DIRECTORY = 'data'

data_location = sess.upload_data(WORK_DIRECTORY, key_prefix=prefix)

print(data_location)

s3://sagemaker-ap-northeast-1-925889618331/LAB-scikit-iris


##  モデルの学習

SageMakerではアルゴリズムの学習する  `Estimator` を作成しまｓ。この  `Estimator` に学習に必要な定義をして `fit` を実行すると学習コンテナが起動して学習を開始します。

Estimaterの引数：
*  __container name__  ECRにあるコンテナを指定します。ハンズオンで作成してコンテナを使います。
*  __role__    Amazon SageMaker がユーザーに代わってタスクを実行するロール.
*  __instance count__   モデルのトレーニングに使用される ML コンピューティングインスタンスの数.
*  __instance type__   モデルのトレーニングに使用される ML コンピューティングインスタンスのタイプ.
*  __output path__    トレーニング結果を保存する S3 へのパス
*  __session__    トレーニングジョブで使用されるその他の AWS サービスとのやり取りを管理するセッションオブジェクト


### VPC の設定
学習コンテナをプライベートVPC内で実行するには、 Estimatorのパラメーターにサブネットとセキュリティグループを指定します。

例

 * subnets=['subnet-002b8823c9926b248','subnet-0bb14b8511a1bfcb3']
 * security_group_ids=['sg-0c88910b00bc4c4b2'] 

In [10]:
account = sess.boto_session.client('sts').get_caller_identity()['Account']
region = sess.boto_session.region_name
image = '{}.dkr.ecr.{}.amazonaws.com/sagemaker-decision-trees:latest'.format(account, region)
print (region)
print (image)

ap-northeast-1
925889618331.dkr.ecr.ap-northeast-1.amazonaws.com/sagemaker-decision-trees:latest


In [11]:
tree = sage.estimator.Estimator(image,
                       role, 1, 'ml.c4.2xlarge',
                       output_path="s3://{}/output".format(sess.default_bucket()),
                       sagemaker_session=sess,
                       subnets=['subnet-0d7e6c04cde7cd82d','subnet-0a6d78ab65e4da2c0'],  ## 要更新 ##
                       security_group_ids=['sg-06828a518846daed5']                       ## 要更新 ##   
                               )

In [12]:
tree.fit(data_location)

2019-08-12 08:19:34 Starting - Starting the training job...
2019-08-12 08:19:36 Starting - Launching requested ML instances......
2019-08-12 08:20:39 Starting - Preparing the instances for training...
2019-08-12 08:21:27 Downloading - Downloading input data
2019-08-12 08:21:27 Training - Downloading the training image...
2019-08-12 08:22:02 Uploading - Uploading generated training model
2019-08-12 08:22:02 Completed - Training job completed

Starting the training.
Training complete.
Billable seconds: 42


## モデルのホスティング

### 推論エンドポイントのVPC設定
ホスティング用のプライベートVPCを設定するには、VpcConfigパラメーターにサブネットとセキュリティグループを指定します。 サブネットはAZの異なる複数が必要です。

VPC configの設定例

  vpc_config_override=
          {'Subnets':['subnet-0931f9262f9680ba4','subnet-019763bdd150fb74e'], 
           'SecurityGroupIds':['sg-0a0612496ec95007a'] }

### 推論エンドポイントのデプロイ

推論を行うために学習したモデルをデプロイします。deploy() メソッドでは、デプロイ先エンドポイントのインスタンス数、インスタンスタイプを指定します。こちらもインスタンスタイプを local にすることで，このインスタンス内にエンドポイントを作成します。

In [13]:
from sagemaker.predictor import csv_serializer
predictor = tree.deploy(1, 'ml.m4.xlarge', serializer=csv_serializer,
                        vpc_config_override={'Subnets':['subnet-0d7e6c04cde7cd82d','subnet-0a6d78ab65e4da2c0'],  ## 要更新 ##
                                                         'SecurityGroupIds':['sg-06828a518846daed5'] })　　　　　## 要更新 ##

---------------------------------------------------------------------------------------!

### 推論の実行

In [14]:
shape=pd.read_csv("data/iris.csv", header=None)
shape.sample(3)

,0,1,2,3,4
36,setosa,5.5,3.5,1.3,0.2
119,virginica,6.0,2.2,5.0,1.5
14,setosa,5.8,4.0,1.2,0.2


In [15]:
# drop the label column in the training set
shape.drop(shape.columns[[0]],axis=1,inplace=True)
shape.sample(3)

,1,2,3,4
41,4.5,2.3,1.3,0.3
25,5.0,3.0,1.6,0.2
77,6.7,3.0,5.0,1.7


In [16]:
import itertools

a = [50*i for i in range(3)]
b = [40+i for i in range(10)]
indices = [i+j for i,j in itertools.product(a,b)]

test_data=shape.iloc[indices[:-1]]

#### 予測モデルに推論APIでリクエスト

In [17]:
print(predictor.predict(test_data.values).decode('utf-8'))

setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
virginica
virginica
virginica
virginica
virginica
virginica
virginica
virginica
virginica



### Cleanup
エンドポイントのデプロイ中はコストが常時発生します。　テストが終わったらエンドポイントを削除してコストを抑えましょう

In [21]:
sess.delete_endpoint(predictor.endpoint)

ClientError: An error occurred (ValidationException) when calling the DeleteEndpoint operation: Could not find endpoint "arn:aws:sagemaker:ap-northeast-1:925889618331:endpoint/sagemaker-decision-trees-2019-08-12-08-19-34-499".

## Batch Transform によるジョブ実行
入力データをS3に置き、バッチジョブで推論を実行し、その結果をS3に出力します。

### Transform ジョブの作成
`Transformer` を作成します。

*  __instance count__ 推論のインスタンス数
*  __instance type__ 推論のインスタンスタイプ
*  __output path__　推論結果のS3の出力先

In [22]:
transform_output_folder = "batch-transform-output"
output_path="s3://{}/{}".format(sess.default_bucket(), transform_output_folder)

transformer = tree.transformer(instance_count=1,
                               instance_type='ml.m4.xlarge',
                               output_path=output_path,
                               assemble_with='Line',
                               accept='text/csv'
                              )

Using already existing model: sagemaker-decision-trees-2019-08-12-08-19-34-499



* The __data_location__ 入力データのS3の場所
* The __content_type__ 入力データのMIMEタイプ
* The __split_type__ 入力データのデリミタ
* The __input_filter__ 　入力データの中で参照する最初のカラムID。その前のカラムを無視します。

In [23]:
transformer.transform(data_location, content_type='text/csv', split_type='Line', input_filter='$[1:]')
transformer.wait()

..............................................!


参照[CreateTransformJob API](https://docs.aws.amazon.com/sagemaker/latest/dg/API_CreateTransformJob.html)

### 結果出力の確認
S3に出力されたデータを確認します。

In [24]:
s3_client = sess.boto_session.client('s3')
s3_client.download_file(sess.default_bucket(), "{}/iris.csv.out".format(transform_output_folder), '/tmp/iris.csv.out')
with open('/tmp/iris.csv.out') as f:
    results = f.readlines()   
print("Transform results: \n{}".format(''.join(results)))

Transform results: 
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
setosa
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
versicolor
virginica
virginica
virginica
virginica
virginica
virginica
virginica
virginica
